In [1]:
from pathlib import Path

In [2]:
import ethicml
from ethicml.algorithms.inprocess import GPyT, GPyTDemPar, GPyTEqOdds, LR, SVM, Agarwal, Kamiran, Kamishima, LRCV, ZafarEqOpp
from ethicml.evaluators import evaluate_models, CrossValidator, run_metrics
from ethicml.data import Compas, Adult, load_data
from ethicml.metrics import Accuracy, ProbPos, TPR, TNR, AbsCV
from ethicml.preprocessing import train_test_split
from ethicml.visualisation.plot import plot_mean_std_box

In [3]:
# YOU DON'T HAVE TO UNDERSTAND THE CODE IN THIS CELL
# we only tell the model where the "run.py" is and where the python executable is
code_dir = Path('..')
def gp(**kwargs):
    return GPyT(code_dir=code_dir, **kwargs)
def gp_dp(**kwargs):
    return GPyTDemPar(code_dir=code_dir, **kwargs)
def gp_eo(**kwargs):
    return GPyTEqOdds(code_dir=code_dir, **kwargs)

In [4]:
# tnr_race_False = 0.724
# tnr_race_True = 0.702
# tnr_sex_True = 0.724
# tnr_sex_False = 0.744
tnr_in_true_race = 0.71
tnr_in_false_race = 0.74
tnr_in_true_sex = 0.72
tnr_in_false_sex = 0.77

In [9]:
# specify flags for GP
gp_flags = dict(epochs=70, length_scale=1.2, use_loo=False, iso=False)

In [10]:
algos = []

algos += [gp(s_as_input=True, flags=gp_flags)]
algos += [gp(s_as_input=False, flags=gp_flags)]
# algos += [gp_dp(epochs=70, s_as_input=True)]

for tpr in [0.6,0.7,0.8,0.9]:
    algos += [gp_eo(s_as_input=True, tnr1=tnr_in_true_race, tnr0=tnr_in_true_race, tpr0=tpr, tpr1=tpr, flags=gp_flags)]
    algos += [gp_eo(s_as_input=False, tnr1=tnr_in_false_race, tnr0=tnr_in_false_race, tpr0=tpr, tpr1=tpr, flags=gp_flags)]

baselines = [
     LR(),
     SVM(),
#     Agarwal(fairness="EqOd"),
#     Kamiran(),
#     Kamishima(),
    ZafarEqOpp(),
]
algos += baselines

In [6]:
data = [
    Compas("Race"),
    Compas("Sex"),
#     Adult("Race"),
#     Adult("Sex"),
]

In [7]:
results = evaluate_models(
    datasets=data,
    inprocess_models=algos,
    metrics=[Accuracy(), ProbPos(), TPR(), TNR()],
    per_sens_metrics=[ProbPos(), TPR(), TNR()], 
    repeats=5,
    delete_prev=True,  # delete previous results
)
results

100%|██████████| 130/130 [38:30<00:00, 11.53s/it, model=ZafarEqOpp, τ=5.0, μ=1.2, dataset=Compas Sex, transform=no_transform, repeat=4]                                   


Accuracy  \
dataset     transform    model                                              repeat              
Compas Sex  no_transform GPyT_in_True                                       0-14460  0.682334   
                         GPyT_in_False                                      0-14460  0.686386   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 0-14460  0.679092   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 0-14460  0.682334   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 0-14460  0.683955   
...                                                                                       ...   
Compas Race no_transform GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 4-12050  0.664506   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 4-12050  0.691248   
                         Logistic Regression, C=1.0                         4-12050  0.678282   
                         SVM                                                4-12050  0.690438   
                         ZafarEqOpp, τ=5.0, μ=1.2                           4-12050  0.662075   

                                                                                          TNR  \
dataset     transform    model                                              repeat              
Compas Sex  no_transform GPyT_in_True                                       0-14460  0.703170   
                         GPyT_in_False                                      0-14460  0.749280   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 0-14460  0.693084   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 0-14460  0.746398   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 0-14460  0.704611   
...                                                                                       ...   
Compas Race no_transform GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 4-12050  0.602305   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 4-12050  0.675793   
                         Logistic Regression, C=1.0                         4-12050  0.757925   
                         SVM                                                4-12050  0.788184   
                         ZafarEqOpp, τ=5.0, μ=1.2                           4-12050  0.680115   

                                                                                     TNR_race_0  \
dataset     transform    model                                              repeat                
Compas Sex  no_transform GPyT_in_True                                       0-14460         NaN   
                         GPyT_in_False                                      0-14460         NaN   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 0-14460         NaN   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 0-14460         NaN   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 0-14460         NaN   
...                                                                                         ...   
Compas Race no_transform GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 4-12050    0.632212   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 4-12050    0.639423   
                         Logistic Regression, C=1.0                         4-12050    0.730769   
                         SVM                                                4-12050    0.745192   
                         ZafarEqOpp, τ=5.0, μ=1.2                           4-12050    0.651442   

                                                                                     TNR_race_0-race_1  \
dataset     transform    model                                              repeat                       
Compas Sex  no_transform GPyT_in_True                        

In [15]:
# import pandas as pd
# results = pd.read_csv("results/Compas Race_no_transform.csv").set_index(["dataset", "transform", "model", "repeat"])

plot_mean_std_box(results, Accuracy(), TPR())
# figs_plots = plot_mean_std_box(results, "Accuracy", "TPR_race_0/race_1")
# figs_plots[0][0]

[(<Figure size 1800x1200 with 1 Axes>,
  <matplotlib.axes._subplots.AxesSubplot at 0x7f82cc336630>),
 (<Figure size 1800x1200 with 1 Axes>,
  <matplotlib.axes._subplots.AxesSubplot at 0x7f82cc269518>)]

In [10]:
train, test = train_test_split(load_data(Adult()))

In [ ]:
# Fair grid search
primary = Accuracy()
fair_measure = AbsCV()
hyperparams = dict(C=[1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7])
lr_cv = CrossValidator(LR, hyperparams, folds=5)
lr_cv.run(train, measures=[primary, fair_measure])
lr_cv.best_hyper_params(primary)

In [ ]:
lr_best_acc = lr_cv.results.get_best_result(primary)
lr_best_fair = lr_cv.results.get_best_in_top_k(primary, fair_measure, top_k=3)
print("best accuracy:", lr_best_acc)
print("best fair(+accuracy):", lr_best_fair)

In [ ]:
lr_best_acc_model = LRCV()
results = evaluate_models(
    datasets=[Adult()],
    inprocess_models=[LR(**lr_best_fair.params), lr_best_acc_model],
    metrics=[Accuracy(), ProbPos(), TPR(), TNR()],
    per_sens_metrics=[ProbPos(), TPR(), TNR()], 
    repeats=3,
    delete_prev=True,  # delete previous results
)

In [ ]:
plot_mean_std_box(results, Accuracy(), ProbPos())